In [2]:
import pandas as pd

In [2]:
pd.__version__

'2.2.2'

In [3]:
df= pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df.head()
df.to_csv('yellow_tripdata_2023-01.csv', index=False)

In [9]:
#How many columns are there?
print(len(df.columns))


19


In [17]:
#What's the standard deviation of the trips duration in January?
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

In [18]:
print(df['duration'].std())

42.594351241920904


In [31]:
#What fraction of the records left after you dropped the outliers?
df_clean = df[(df['duration'] >= 1) & (df['duration'] <= 60)]
fraction = len(df_clean) / len(df)
print(f"Fraction of records kept: {fraction:.2%}")


Fraction of records kept: 98.12%


In [25]:
#What's the dimensionality of this matrix (number of columns)?
df_clean = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

In [27]:
df_clean = df_clean.copy()  # explicitly make a copy to avoid the warning
df_clean.loc[:, 'PULocationID'] = df_clean['PULocationID'].astype(str)
df_clean.loc[:, 'DOLocationID'] = df_clean['DOLocationID'].astype(str)

In [28]:
dicts = df_clean[['PULocationID', 'DOLocationID']].to_dict(orient='records')

In [29]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer()
X = dv.fit_transform(dicts)

In [30]:
print(X.shape)

(3009173, 515)


In [40]:
#What's the RMSE on train?
# Step 4: DictVectorizer
from sklearn.feature_extraction import DictVectorizer
dicts = df_clean[['PULocationID', 'DOLocationID']].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(dicts)

# Step 5: Train model
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, df_clean['duration'])

# Step 6: Predict & evaluate
from sklearn.metrics import mean_squared_error
import numpy as np
y_pred = model.predict(X_train)
rmse = np.sqrt(mean_squared_error(df_clean['duration'], y_pred))
print(f"RMSE on train: {rmse:.2f}")

RMSE on train: 7.65
